In [ ]:
# Install Dependencies
!pip install -U langchain langchain-community langchain-huggingface sentence-transformers chromadb pdfplumber openpyxl transformers accelerate unstructured[all] gradio

# Imports
import os
import pdfplumber
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain.schema import Document
from transformers import pipeline
import gradio as gr

# Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-V2")

# Folder for Chroma DB
os.makedirs("./chroma_store", exist_ok=True)

# Document Loading Function
def load_documents(uploaded_files):
    docs = []
    for file_obj in uploaded_files:
        filename = file_obj.name
        ext = filename.split('.')[-1].lower()

        try:
            # PDF Processing
            if ext == "pdf":
                text = ""
                with pdfplumber.open(file_obj.name) as pdf:
                    total_pages = len(pdf.pages)
                    print(f"Reading {filename} ({total_pages} pages)")
                    for i, page in enumerate(pdf.pages):
                        text += page.extract_text() or ""
                docs.append(Document(page_content=text, metadata={"source": filename}))

            # Word Document Processing
            elif ext == "docx":
                loader = UnstructuredWordDocumentLoader(file_obj.name)
                docs.extend(loader.load())

            # Excel File Processing (using pandas)
            elif ext in ["xls", "xlsx"]:
                print(f"Reading Excel file: {filename}")
                xl = pd.ExcelFile(file_obj.name)
                for sheet_name in xl.sheet_names:
                    df = xl.parse(sheet_name)
                    text = df.to_string(index=False)
                    docs.append(Document(page_content=text, metadata={"source": f"{filename} - {sheet_name}"}))

            else:
                print(f"Unsupported file type: {filename}")
                continue

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    return docs


# Vector Store Setup
def create_vector_store(docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = splitter.split_documents(docs)
    vectordb = Chroma.from_documents(texts, embedding_model, persist_directory="./chroma_store")
    vectordb.persist()
    return vectordb


# Build RAG Pipeline
def build_qa_chain(vectordb):
    from langchain.chains import RetrievalQA
    from langchain.llms import HuggingFacePipeline

    flan_pipe = pipeline("text2text-generation", model="google/flan-t5-large", device=-1)
    flan_llm = HuggingFacePipeline(pipeline=flan_pipe)

    retriever = vectordb.as_retriever(search_kwargs={"k": 3})
    qa_chain = RetrievalQA.from_chain_type(
        llm=flan_llm,
        retriever=retriever,
        return_source_documents=True
    )
    return qa_chain


# Gradio Chat Logic
qa_chain = None

def process_files(files):
    global qa_chain
    docs = load_documents(files)
    vectordb = create_vector_store(docs)
    qa_chain = build_qa_chain(vectordb)
    return f"Loaded {len(docs)} documents and initialized RAG system!"

def chat_with_docs(message, history):
    global qa_chain
    if message.lower() == "exit":
        return "Exiting chat. Refresh to restart."
    if not qa_chain:
        return "Please upload and process documents first."
    result = qa_chain(message)
    sources = "\n".join([f"- {src.metadata['source']}" for src in result["source_documents"]])
    return f"**Answer:** {result['result']}\n\n**Sources:**\n{sources}"


# Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("RAG_bot (PDF, DOCX, XLSX)")
    file_input = gr.File(file_count="multiple", label="Upload your documents")
    process_button = gr.Button("Process Documents")
    status_output = gr.Textbox(label="Status", interactive=False)

    chatbot = gr.ChatInterface(fn=chat_with_docs, title="Chat with your documents")

    process_button.click(process_files, inputs=file_input, outputs=status_output)

app.launch()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.9 MB/s eta 0:00:00
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.4
    Uninstalling pydantic_core-2.41.4:
      Successfully uninstalled pydantic_core-2.41.4
  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 25.1.0
    Uninstalling aiofiles-25.1.0:
      Successfully uninstalled aiofiles-25.1.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.2
    Uninstalling pydantic-2.12.2:
      Successfully uninstal

/tmp/ipython-input-3682193478.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-V2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8f5df49715b5445f9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
